In [ ]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.

This is by no means production code.
"""

In [1]:
# built-in imports
from time import sleep
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
# from urllib.request import urlopen

import requests
from random import random

!pip install tqdm
from tqdm import tqdm

import csv
import os
# comment
import pandas as pd

In [2]:
property_files = '../data/raw/property_meta'
if not os.path.exists(property_files):
    os.makedirs(property_files)

In [3]:
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppelWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

# constants
BASE_URL = "https://www.domain.com.au"
sort_methods = ["default-desc", "dateupdated-desc", "price-asc", "price-desc", "suburb-asc"]
N_PAGES = range(1, 51) # update this to your liking

In [4]:
url_links = []
property_metadata = defaultdict(dict)

In [5]:
for sort_method in sort_methods:
    for page in tqdm(N_PAGES):
        url = BASE_URL + f"/rent/melbourne-region-vic/?sort={sort_method}&page={page}"
        bs_object = BeautifulSoup(requests.get(url, headers = headers).text, "html.parser")
        # find the unordered list (ul) elements which are the results, then
        # find all href (a) tags that are from the base_url website.
        index_links = bs_object.find("ul",{"data-testid": "results"}).findAll("a",href=re.compile(f"{BASE_URL}/*"))
        for link in index_links:
            # if its a property address, add it to the list
            if 'address' in link['class']:
                url_links.append(link['href'])
        sleep(round(random(),2))

  0%|          | 0/50 [01:39<?, ?it/s]


KeyboardInterrupt: 

In [79]:
len(url_links)

4208

In [80]:
url_links = list(set(url_links))
num_url = len(url_links)

In [81]:
# urls are stored in a csv file
# as property information are updated real time and scraped over a week
# future scraping should be compared to this to avoid duplication
with open(f'{property_files}/property_url.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    for x in url_links:
        csvwriter.writerow([x])

In [131]:
# for each url, scrape some basic metadata
count = 0
for property_url in tqdm(url_links):
    count += 1
    # bs_object = BeautifulSoup(urlopen(url), "lxml")
    bs_object = BeautifulSoup(requests.get(property_url, headers = headers).text, "html.parser")
    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object.find("h1", {"class": "css-164r41r"}).text
    # print(property_metadata[property_url]['name'])
    property_metadata[property_url]['type'] = bs_object.find("div", {"data-testid": "listing-summary-property-type"}).text
    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object.find("div", {"data-testid": "listing-details__summary-title"}).text
    # print(property_metadata[property_url]['cost_text'])
    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object
                .find(
                "a",
                {"target": "_blank", 'rel': "noopener noreferer"}
            )
                .attrs['href']
        )[0].split(',')
    ]
    # print(property_metadata[property_url]['coordinates'])
    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object
            .find("div", {"data-testid": "property-features"})
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    # print(property_metadata[property_url]['rooms'])
    property_metadata[property_url]['desc'] = re.sub(r'<br\/>', '\n', str(bs_object.find("p"))).strip('</p>')
    # print(property_metadata[property_url]['desc'])
    # sleep(round(3*random(),2))

  0%|          | 11/4309 [00:06<39:44,  1.80it/s]


KeyboardInterrupt: 

In [86]:
with open(f'{property_files}/property_metadata.json', 'w') as f:
    dump(property_metadata, f)

In [83]:
print(len(property_metadata))

4207


In [125]:
df = pd.DataFrame(property_metadata).T.reset_index()
df = df.rename(columns = {'index':'url'})

In [ ]:
df.head()

In [129]:
df.to_csv(f'{property_files}/property_metadata.csv')